In [16]:
import os
import pandas as pd
import glob
import numpy as np
import librosa
import plotly.graph_objs as go
%matplotlib inline
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

ImportError: Numba needs NumPy 1.20 or less

In [ ]:
""" def set_keras_backend(backend):
    print("A acertar o backend e libertar memória da grafica")
    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        importlib.reload(K)
        assert K.backend() == backend
    if backend == "tensorflow":
        # Allow memory growth for the GPU
        physical_devices = tf.config.experimental.list_physical_devices('GPU')
        print(physical_devices)
        tf.config.experimental.set_memory_growth(physical_devices[0], True)  

    
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

set_keras_backend("tensorflow") """

' def set_keras_backend(backend):\n    print("A acertar o backend e libertar memória da grafica")\n    if K.backend() != backend:\n        os.environ[\'KERAS_BACKEND\'] = backend\n        importlib.reload(K)\n        assert K.backend() == backend\n    if backend == "tensorflow":\n        # Allow memory growth for the GPU\n        physical_devices = tf.config.experimental.list_physical_devices(\'GPU\')\n        print(physical_devices)\n        tf.config.experimental.set_memory_growth(physical_devices[0], True)  \n\n    \ndef limit_mem():\n    K.get_session().close()\n    cfg = K.tf.ConfigProto()\n    cfg.gpu_options.allow_growth = True\n    K.set_session(K.tf.Session(config=cfg))\n\nset_keras_backend("tensorflow") '

In [ ]:
#PATHS
PATH_CSV = './features.csv'
CLASSES_NAMES=['negative', 'positive', 'unknown']

In [ ]:
data=pd.read_csv(PATH_CSV)
data.head()

,Unnamed: 0,filename,label,chroma_stft,mel_spectogram,spectral_contrast,tonnetz,mfcc1,mfcc2,mfcc3,...,mfcc31,mfcc32,mfcc33,mfcc34,mfcc35,mfcc36,mfcc37,mfcc38,mfcc39,mfcc40
0,0,4-164206-A-10.wav,10,0.868969,0.861374,15.341950,-0.000752,-48.946632,55.206245,-7.047971,...,-3.782364,1.401003,-2.568921,2.393073,-1.468146,2.555113,-1.484496,2.290247,-2.209650,1.044895
1,1,2-83270-A-13.wav,13,0.218590,0.031675,23.631527,0.027424,-549.716125,-63.588486,-38.799469,...,-17.556175,13.050779,-5.685339,0.622180,-2.255047,-0.585254,4.197667,-6.788848,5.966758,-6.047862
2,2,3-110913-A-7.wav,7,0.658248,0.830945,19.271990,-0.004003,-201.649429,101.756508,6.351685,...,-8.344519,-4.245569,-10.650942,-5.038177,-5.601541,-0.002323,-5.730710,-3.703166,-6.499656,-0.611325
3,3,4-144083-A-16.wav,16,0.785159,2.127163,16.364365,-0.003129,-145.423508,181.667603,-20.035183,...,-4.562034,0.138662,-3.261474,1.118976,-1.880353,2.038065,-0.930130,2.180940,-1.298151,0.926360
4,4,3-51376-A-42.wav,42,0.336802,13.715096,21.548095,-0.013215,-179.293137,170.124237,-58.605957,...,6.506222,1.723845,-5.902114,-5.278167,-1.353401,-0.057298,2.746562,4.565757,-3.365773,-6.954345


In [ ]:
def new_column(csv_file):
    label_2 = pd.Series([]) 
  
    for i in range(len(data)): 
        if data['label'][i] == 0: 
            label_2[i]='negative'
  
        elif data['label'][i] == 7: 
            label_2[i]='unknown'
  
        elif data['label'][i] == 10: 
            label_2[i]='unknown'
        
        elif data['label'][i] == 11: 
            label_2[i]='unknown'
    
        elif data['label'][i] == 13: 
            label_2[i]='unknown'
        
        elif data['label'][i] == 14: 
            label_2[i]='unknown'
    
        elif data['label'][i] == 16: 
            label_2[i]='unknown'
        
        elif data['label'][i] == 20: 
            label_2[i]='negative'
        
        elif data['label'][i] == 36: 
            label_2[i]='negative'
    
        elif data['label'][i] == 42: 
            label_2[i]='negative'
        
        elif data['label'][i] == 48: 
            label_2[i]='negative'
        
        elif data['label'][i] == 51: 
            label_2[i]='positive'
        
        elif data['label'][i] == 52: 
            label_2[i]='positive'
  
        else: 
            label_2[i]= data['label'][i] 
  
    # inserting new column with values of list made above         
    data.insert(2, 'label_2', label_2) 
    
new_column(data)

In [ ]:
data.head()

,Unnamed: 0,filename,label_2,label,chroma_stft,mel_spectogram,spectral_contrast,tonnetz,mfcc1,mfcc2,...,mfcc31,mfcc32,mfcc33,mfcc34,mfcc35,mfcc36,mfcc37,mfcc38,mfcc39,mfcc40
0,0,4-164206-A-10.wav,unknown,10,0.868969,0.861374,15.341950,-0.000752,-48.946632,55.206245,...,-3.782364,1.401003,-2.568921,2.393073,-1.468146,2.555113,-1.484496,2.290247,-2.209650,1.044895
1,1,2-83270-A-13.wav,unknown,13,0.218590,0.031675,23.631527,0.027424,-549.716125,-63.588486,...,-17.556175,13.050779,-5.685339,0.622180,-2.255047,-0.585254,4.197667,-6.788848,5.966758,-6.047862
2,2,3-110913-A-7.wav,unknown,7,0.658248,0.830945,19.271990,-0.004003,-201.649429,101.756508,...,-8.344519,-4.245569,-10.650942,-5.038177,-5.601541,-0.002323,-5.730710,-3.703166,-6.499656,-0.611325
3,3,4-144083-A-16.wav,unknown,16,0.785159,2.127163,16.364365,-0.003129,-145.423508,181.667603,...,-4.562034,0.138662,-3.261474,1.118976,-1.880353,2.038065,-0.930130,2.180940,-1.298151,0.926360
4,4,3-51376-A-42.wav,negative,42,0.336802,13.715096,21.548095,-0.013215,-179.293137,170.124237,...,6.506222,1.723845,-5.902114,-5.278167,-1.353401,-0.057298,2.746562,4.565757,-3.365773,-6.954345


In [ ]:
def load_data(data):  
    X = data.drop([data.columns[0],'filename', 'label', 'label_2'],axis=1) #droping first column, filename column and label column
    Y= data['label_2'] 
    print('X:', X.shape)
    print('Y:', Y.shape)
    return (X, Y)

(X, Y) = load_data(data)

X: (519, 44)
Y: (519,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=1)

print('X_train', X_train.shape)
print("X_test:", X_test.shape)
print("y_train:", y_train.shape)
print("y_test:", y_test.shape)

X_train (415, 44)
X_test: (104, 44)
y_train: (415,)
y_test: (104,)


In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
Y_train = enc.fit_transform(y_train.values.reshape(-1,1)).toarray()
Y_test = enc.fit_transform(y_test.values.reshape(-1,1)).toarray()
train = np.argmax(Y_train,axis =  1)
test = np.argmax(Y_test, axis = 1)
print('X_test:', X_test.shape)
print('X_train:', X_train.shape)
print('Y_train:', Y_train.shape)
print('Y_test:', Y_test.shape)
print('train:', train.shape)
print('test:', test.shape)

X_test: (104, 44)
X_train: (415, 44)
Y_train: (415, 3)
Y_test: (104, 3)
train: (415,)
test: (104,)


In [ ]:
clf = XGBClassifier(learning_rate=0.1,
                    n_estimators=400,
                    max_depth=5,
                    min_child_weight=1,
                    gamma=0,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='multi:softprob',
                    nthread=4,
                    num_class=3,
                    seed=27,
                    predictor = 'gpu_predictor',
                    tree_method='gpu_hist',
                    gpu_id=0)


clf.fit(X_train,train.ravel(),
            verbose=False,
            early_stopping_rounds=50,
            eval_metric='merror',
            eval_set=[(X_test, test.ravel())])

y_predict = clf.predict(X_test)
y_train_predict = clf.predict(X_train)

print(f'Training accuracy:        {accuracy_score(train,  y_train_predict)*100: .2f}%')
print(f'Validation accuracy:      {accuracy_score(test, y_predict)*100: .2f}%')

clf.save_model("model.json")



Training accuracy:         100.00%
Validation accuracy:       92.31%


In [ ]:
model = XGBClassifier()
model = model.load_model("model.json")

In [ ]:
from pydub import AudioSegment

os.makedirs('./classical_timestamps', exist_ok=True)

audio_file= "./classical.00010.wav"
audio = AudioSegment.from_wav(audio_file)
list_of_timestamps = [5, 10, 15, 20, 25, 30] #and so on in *seconds*

start = 0
for  idx,t in enumerate(list_of_timestamps):
    #break loop if at last element of list
    if idx == len(list_of_timestamps):
        break

    end = t * 1000 #pydub works in millisec
    print("split at [ {}:{}] ms".format(start, end))
    audio_chunk=audio[start:end]
    audio_chunk.export( "./classical_timestamps/classical_{}.wav".format(end), format="wav") 

    start = end  #pydub works in millisec

split at [ 0:5000] ms
split at [ 5000:10000] ms
split at [ 10000:15000] ms
split at [ 15000:20000] ms
split at [ 20000:25000] ms
split at [ 25000:30000] ms


In [ ]:
#csv
def create_csv_header():
    header = ['filename','label','chroma_stft','mel_spectogram','spectral_contrast','tonnetz']
    for i in range(1, 41):
        header.append(f' mfcc{i}')
    return header


header = create_csv_header()
print(header)

['filename', 'label', 'chroma_stft', 'mel_spectogram', 'spectral_contrast', 'tonnetz', ' mfcc1', ' mfcc2', ' mfcc3', ' mfcc4', ' mfcc5', ' mfcc6', ' mfcc7', ' mfcc8', ' mfcc9', ' mfcc10', ' mfcc11', ' mfcc12', ' mfcc13', ' mfcc14', ' mfcc15', ' mfcc16', ' mfcc17', ' mfcc18', ' mfcc19', ' mfcc20', ' mfcc21', ' mfcc22', ' mfcc23', ' mfcc24', ' mfcc25', ' mfcc26', ' mfcc27', ' mfcc28', ' mfcc29', ' mfcc30', ' mfcc31', ' mfcc32', ' mfcc33', ' mfcc34', ' mfcc35', ' mfcc36', ' mfcc37', ' mfcc38', ' mfcc39', ' mfcc40']


In [ ]:
PATH_CSV_CLASSICAL_TIMESTAMPS = './features_classical_timestamps.csv'
PATH_CLASSICAL_TIMESTAMPS = './classical_timestamps'

In [ ]:
def generate_csv_music(PATH_CLASSICAL_TIMESTAMPS, path_csv,file_ext='*.wav'):
    if os.path.exists(path_csv):
        print(f'CSV {path_csv} already exists')
    else:
        sound_cases=[]
        for fn in glob.glob(os.path.join(PATH_CLASSICAL_TIMESTAMPS, file_ext)): #classical music
            filename = fn.split('\\')[1]
            label = 3
            sound_data=[filename]
            sound_data.append(label)
            sound = f'{PATH_CLASSICAL_TIMESTAMPS}/{filename}'
            X, sr = librosa.load(sound)
            stft = np.abs(librosa.stft(X))
            sound_data.append(np.mean(librosa.feature.chroma_stft(S=stft, sr=sr)))
            sound_data.append(np.mean(librosa.feature.melspectrogram(X, sr=sr)))
            sound_data.append(np.mean(librosa.feature.spectral_contrast(S=stft, sr=sr)))
            sound_data.append(np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sr)))
            mfcc = librosa.feature.mfcc(y=X, sr=sr, n_mfcc=40)
            for e in mfcc:
                sound_data.append(np.mean(e))
            sound_cases.append(sound_data)
                
        df = pd.DataFrame(sound_cases, columns = header) 
        df.to_csv(path_csv)
        
generate_csv_music(PATH_CLASSICAL_TIMESTAMPS, PATH_CSV_CLASSICAL_TIMESTAMPS)

NotImplementedError: Cannot load filter definition for kaiser_fast

In [ ]:
DATA_3=pd.read_csv(PATH_CSV_CLASSICAL_TIMESTAMPS)
DATA_3.head()

,Unnamed: 0,filename,label,chroma_stft,mel_spectogram,spectral_contrast,tonnetz,mfcc1,mfcc2,mfcc3,...,mfcc31,mfcc32,mfcc33,mfcc34,mfcc35,mfcc36,mfcc37,mfcc38,mfcc39,mfcc40
0,0,classical_10000.wav,3,0.436572,0.431586,22.674332,0.010457,-272.59976,126.85941,-37.440510,...,-1.727285,-8.259519,-1.804429,-1.633463,2.685548,-2.607638,-3.702459,0.839746,-7.673559,-13.245373
1,1,classical_15000.wav,3,0.434985,0.048627,21.159292,-0.007166,-411.93182,137.60664,-28.331543,...,3.484850,-2.130914,-5.943365,-5.748062,-3.888671,-3.459251,-0.457856,0.749646,5.714317,5.458793
2,2,classical_20000.wav,3,0.398473,0.439391,23.392819,-0.031331,-270.98510,129.79470,-33.484905,...,10.258470,-8.598377,-8.882826,-2.746100,-0.378583,0.181252,3.426939,-5.152680,-0.415268,4.147439
3,3,classical_25000.wav,3,0.422367,0.338780,23.293577,0.014714,-273.61087,132.17610,-39.424362,...,6.193310,-4.851325,-5.776797,-5.848343,-4.897565,-2.433606,-2.553839,-2.885394,6.430138,0.236686
4,4,classical_30000.wav,3,0.367448,0.338380,24.099028,-0.006981,-284.87933,126.03677,-42.831450,...,0.748452,-6.972674,-2.332934,-5.582502,-1.716171,-1.201393,1.665013,1.852122,-2.217466,-1.797750


In [ ]:
def load_data(data):  
    X_TEST_CLASSICAL = data.drop([data.columns[0],'filename', 'label'],axis=1)#droping first column, filename column and label column 
    Y_TEST_CLASSICAL = data['label']
    print('X_TEST_CLASSICAL:', X_TEST_CLASSICAL.shape)
    print('Y_TEST_CLASSICAL:', Y_TEST_CLASSICAL.shape)
    return (X_TEST_CLASSICAL, Y_TEST_CLASSICAL)

(X_TEST_CLASSICAL,Y_TEST_CLASSICAL) = load_data(DATA_3)

X_TEST_CLASSICAL: (6, 44)
Y_TEST_CLASSICAL: (6,)


In [ ]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return indices

In [ ]:
def truncate(n, decimals=0):
    multiplier = 10 ** decimals
    return int(n * multiplier) / multiplier

In [ ]:
def print_series_prediction_3(sound, label):
    sounds = []
    audio = []
    first_preds = []
    first_percent = []
    second_preds = []
    second_percent = []
    third_preds = []
    third_percent = []
    for i in range(len(sound)): #para imprimir tabela de previsoes
        predictions = model.predict_proba(sound)
        predic = largest_indices(predictions[i], 3)
        percentage = predictions[i][predic]
        
        sounds.append(label[i])
        #audio.append(display(Audio(AUDIO, autoplay=True, rate=rate)))
        first_preds.append((predic)[0])
        first_percent.append(truncate(percentage[0],3))
        second_preds.append((predic)[1])
        second_percent.append(truncate(percentage[1],3))
        third_preds.append((predic)[2])
        third_percent.append(truncate(percentage[2],3))
        mean_first = sum(first_preds)/ len(first_preds)
        mean_first = int(round(mean_first))
        mean_second = sum(second_preds)/ len(second_preds)
        mean_second = int(round(mean_second))
        mean_third = sum(third_preds)/ len(third_preds)
        mean_third = int(round(mean_third))
       
    data = {'Y_test':  sounds, #'Sound': audio, 'mean_first' : mean_first, 'mean_second' : mean_second,'mean_third': mean_third 
            'first_pred': first_preds,
            'first_percent': first_percent,
            'second_pred': second_preds,
            'second_percent': second_percent,
            'third_pred': third_preds,
            'third_percent': third_percent
           }
    
    df = pd.DataFrame(data)
    fig = go.Figure(data=[go.Table(
    header=dict(values=list(df.columns),
                fill_color='lavender',
                align='center'),
    cells=dict(values=[df.Y_test, df.first_pred, df.first_percent, df.second_pred, # df.Sound,df.mean_first,df.mean_second,df.mean_third
                       df.second_percent, df.third_pred, df.third_percent],
               #fill_color='lightgrey',
               fill=dict(color=['lightgrey', 'white']),
               align='center'))])

    fig.show()
    
    print('Mean of timestamps:', mean_first)
    
    return df

df = print_series_prediction_3(X_TEST_CLASSICAL, Y_TEST_CLASSICAL)

Mean of timestamps: 2
